# Projet maison n° 3

In [2]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [165]:
# names us
def df_names_us():
    
    years=list(range(1880,2021))
    files = ["yob"+str(year)+".txt" for year in years]
    df=pd.DataFrame(columns=["year","name","gender","births"])

    for file in files:
        temp = pd.read_csv(file,header=None,names=["name","gender","births"])
        temp["year"]=file[3:7]
        df=df.append(temp)
        
    df.year=df.year.astype(int)
    df.births = df.births.astype(int)
    df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])
    df=df.reset_index(drop=True)
    return df

In [163]:
a=df_names_us()

In [164]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2020863 entries, 0 to 31270
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   year    int64 
 1   name    object
 2   gender  object
 3   births  int64 
dtypes: int64(2), object(2)
memory usage: 77.1+ MB


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [185]:
# names fr
def df_names_fr():
    df = pd.read_csv('nat2020.csv', delimiter=";", skiprows = 1,header=None, 
                     names=['gender', 'name', 'year', 'births'],
                     dtype={'gender': 'int', 'name': 'string', 'births': 'int'})
    
    df['gender'] = df['gender'].map({2: 'F', 1: 'M'})

    df = df[df.name != "_PRENOMS_RARES"]
    df['name'] = df['name'].apply(str.capitalize)
    df= df[df['name'].apply(len)>=2]
    
    df = df[df.year != "XXXX"]
    df.year = df.year.apply(int)
    
    df = df[['year', 'name', 'gender', 'births']]
    df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, False, False, True])
    
    df = df.reset_index(drop=True)
    return df

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [152]:
# taux de change
def df_taux_change(devises):

    df = pd.read_csv('Webstat_Export_20210929.txt', header=0, skiprows=[0,2,3,4,5], sep=';', decimal=',',
    usecols=['Code série :'] + ['EXR.D.'+d+'.EUR.SP00.A' for d in devises])

    df =  df.rename(columns={**{'Code série :':'date'}, **{'EXR.D.'+d+'.EUR.SP00.A':d for d in devises}})

    df['date'] = df['date'].apply(lambda x: pd.to_datetime(x, dayfirst=True))

    df = df.set_index('date')

    df = df.apply(lambda x: pd.to_numeric(x.str.replace(',', '.'), errors='coerce'))

    df = df.dropna()
    
   
    return df

# Tests

In [182]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630406)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [183]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [186]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 25.240s

OK
